In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import json

from torch.utils.data import Dataset,DataLoader
import torch
from torch import nn
from torch.nn.modules.loss import _WeightedLoss
import timm
import time
from tqdm import tqdm_notebook as tqdm
import cv2
import numpy as np
#from torch.cuda.amp import autocast, GradScaler

import matplotlib.pyplot as plt
import random

%matplotlib inline

In [3]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)
from albumentations.pytorch import ToTensorV2

import apex

In [4]:
test_fold = 0
#img_size = 512
#bs = 16
epochs_to_predict = [7,8,9]
LOG_FILE = f"distil_03_07.log"

In [5]:
CFG = {
    'fold_num': 0,
    'seed': 2021,
    'model_arch': 'tf_efficientnet_b4_ns',
    'img_size': 512,
    'epochs': 10,
    'train_bs': 16,
    'valid_bs': 32,
    'T_0': 10,
    'lr': 1e-4,
    'min_lr': 1e-6,
    'weight_decay':1e-6,
    'num_workers': 8,
    'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'fp16': True
}

In [6]:
TRAIN_AUGS = Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
VAL_AUGS =  Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)
TEST_AUGS =  Compose([
            Resize(CFG['img_size'], CFG['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)
# Worth trying: Blur, Noize,

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

seed_everything(2021)

In [8]:
data = pd.read_csv('/home/data/Cassava/train.csv')
print(data.shape)
# data.label.hist();

(21397, 2)


In [9]:
mapping = json.load(open('/home/data/Cassava/label_num_to_disease_map.json', 'r'))

In [10]:
data['fold'] = 0
#add random state
strkf = StratifiedKFold(n_splits=5)
_ =strkf.get_n_splits(data.image_id, data.label)
f = 0
for train_index, test_index in strkf.split(data.image_id, data.label):
    #print("TRAIN:", train_index, "TEST:", test_index)
    data.loc[data.index.isin(test_index), 'fold'] = f 
    f = f+1
data.fold.value_counts()

1    4280
0    4280
4    4279
3    4279
2    4279
Name: fold, dtype: int64

In [11]:
class CassavaDataset(Dataset):
    def __init__(self, df, mode = 'train'):
        super().__init__()
        self.df = df
        self.mode = mode
        
    def __len__(self, ):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df[self.df.index == idx]
        image_name = row.image_id.values[0]
        img = get_img('/home/data/Cassava/train_images/'+ image_name)
        if self.mode == 'train':
            label = row.label.values[0]
            img = TRAIN_AUGS(image=img)['image']
            #img = np.moveaxis(img, 2, 0)
            return img, label
        elif self.mode == 'val':
            label = row.label.values[0]
            img = VAL_AUGS(image=img)['image']
            return img, label
        elif self.mode == 'test':
            img = TEST_AUGS(image=img)['image']
            return img
        else:
            print("Unknown mod type")

In [12]:
def train_one_epoch(model, optim, train_loader, loss_fn, epoch):
    model = model.train();
    t = time.time()
    running_loss = None
    preds_all = []
    targets_all = []
    loss_sum = 0
    sample_num = 0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), position=0, leave=True)
    for step, (x, y_true) in pbar:
        x = x.to(device).float()
        y_true = y_true.to(device).long()
        y_pred = model(x)
        l = loss_fn(y_pred, y_true)
        optim.zero_grad()
        l.backward()
        optim.step()
        preds_all += [torch.argmax(y_pred, 1).detach().cpu().numpy()]
        targets_all += [y_true.detach().cpu().numpy()]
        if running_loss is None:
            running_loss = l.item()
        else:
            running_loss = running_loss * .99 + l.item() * .01

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
            description = f'tain epoch {epoch} loss: {running_loss:.4f}'
            pbar.set_description(description)
    
    if scheduler is not None:
        scheduler.step()
    preds_all = np.concatenate(preds_all)
    targets_all = np.concatenate(targets_all)
    print("Target acc = ", (preds_all==targets_all).mean())
    with open(LOG_FILE, 'a+') as logger:
        logger.write(f"Epoch # {epoch}, train acc = {(preds_all==targets_all).mean()}, ")

In [13]:
def valid_one_epoch(model, optim, val_loader, loss_fn, epoch):
    preds_all = []
    t = time.time()
    loss_sum = 0
    sample_num = 0
    preds_all = []
    targets_all = []
    pbar = tqdm(enumerate(val_loader), total=len(val_loader))
    with torch.no_grad():
        model = model.eval();
        for step, (x, y_true) in pbar:
            x = x.to(device).float()
            y_true = y_true.to(device).long()
            y_pred = model(x)
            preds_all += [torch.argmax(y_pred, 1).detach().cpu().numpy()]
            targets_all += [y_true.detach().cpu().numpy()]
            l = loss_fn(y_pred, y_true)
            loss_sum += l.item()*y_true.shape[0]
            sample_num += y_true.shape[0]  

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            description = f'val epoch {epoch} loss: {loss_sum/sample_num:.4f}'
            pbar.set_description(description)
    
    preds_all = np.concatenate(preds_all)
    targets_all = np.concatenate(targets_all)
    print('validation multi-class accuracy = {:.4f}'.format((preds_all==targets_all).mean()))
    with open(LOG_FILE, 'a+') as logger:
        logger.write(f"val acc = {(preds_all==targets_all).mean()}\n")
    return (preds_all==targets_all).mean(), loss_sum/sample_num

In [14]:
class CassvaImgClassifier(torch.nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        '''
        self.model.classifier = nn.Sequential(
            nn.Dropout(0.3),
            #nn.Linear(n_features, hidden_size,bias=True), nn.ELU(),
            nn.Linear(n_features, n_class, bias=True)
        )
        '''
    def forward(self, x):
        x = self.model(x)
        return x

In [15]:
device = torch.device('cuda')

In [16]:

# for dev_fold in [1,2,3,4]:
#     train_data = data[(data.fold != test_fold) & (data.fold != dev_fold)].reset_index(drop=True)
#     val_data = data[data.fold == test_fold].reset_index(drop=True)
#     dev_data = data[data.fold == dev_fold].reset_index(drop=True)  
    
#     train_ds = CassavaDataset(train_data, mode = 'train')
#     dev_ds = CassavaDataset(val_data, mode = 'val')
#     valid_ds = CassavaDataset(val_data, mode = 'val')
    
#     train_loader = torch.utils.data.DataLoader(
#         train_ds,
#         batch_size=CFG['train_bs'],
#         pin_memory=False,
#         drop_last=False,
#         shuffle=True,        
#         num_workers=8)
    
#     val_loader = torch.utils.data.DataLoader(
#         valid_ds, 
#         batch_size=CFG['valid_bs'],
#         num_workers=8,
#         shuffle=False,
#         pin_memory=False)
    
#     dev_loader = torch.utils.data.DataLoader(
#         dev_ds, 
#         batch_size=CFG['valid_bs'],
#         num_workers=8,
#         shuffle=False,
#         pin_memory=False)
    
#     model = CassvaImgClassifier('tf_efficientnet_b4_ns', data.label.nunique(), pretrained=True).to(device)
    
#     optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4, weight_decay=1e-6)
#     scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['epochs'], T_mult=1, eta_min=1e-6, last_epoch=-1)
    
#     loss_tr = nn.CrossEntropyLoss().to(device)
#     loss_fn = nn.CrossEntropyLoss().to(device)
#     for epoch in range(CFG['epochs']):
#         print(f"epoch number {epoch}, lr = {optimizer.param_groups[0]['lr']}")
#         train_one_epoch(model, optimizer, train_loader, loss_tr, epoch)
#         val_acc, val_loss = valid_one_epoch(model, optimizer, val_loader, loss_fn, epoch)

#         torch.save(model.state_dict(),'./output/{}_dev_fold_{}_test_fold_{}_epoch_{}_val_loss_{:.4f}_val_acc_{:.4f}'.format(CFG['model_arch'], dev_fold, test_fold, epoch, val_acc, val_loss))

#     #del model, optimizer, train_loader, val_loader, scaler, scheduler
#     torch.cuda.empty_cache()

In [17]:
#distil

In [18]:
PATH = [
        "/home/samenko/Cassava/output/tf_efficientnet_b4_ns_dev_fold_1_test_fold_0_epoch_5_val_loss_0.8820_val_acc_0.3677",
        "/home/samenko/Cassava/output/tf_efficientnet_b4_ns_dev_fold_2_test_fold_0_epoch_4_val_loss_0.8825_val_acc_0.3624",
        "/home/samenko/Cassava/output/tf_efficientnet_b4_ns_dev_fold_3_test_fold_0_epoch_6_val_loss_0.8825_val_acc_0.3679",
        "/home/samenko/Cassava/output/tf_efficientnet_b4_ns_dev_fold_4_test_fold_0_epoch_5_val_loss_0.8811_val_acc_0.3739"]

In [19]:
from scipy.special import softmax

In [20]:
# train_data_cp = []
# for dev_fold in [1,2,3,4]:
#     print(f"dev fold: {dev_fold}")
#     dev_data = data[data.fold == dev_fold].reset_index(drop=True)  
#     dev_data_cp = dev_data.copy()
#     dev_ds = CassavaDataset(dev_data, mode = 'val')

    
#     dev_loader = torch.utils.data.DataLoader(
#         dev_ds, 
#         batch_size=CFG['valid_bs'],
#         num_workers=8,
#         shuffle=False,
#         pin_memory=False)    
#     submission = []
#     model = CassvaImgClassifier('tf_efficientnet_b4_ns', data.label.nunique(), pretrained=True).to(device)
#     _ = model.load_state_dict(torch.load(PATH[dev_fold - 1]))
    
#     dev_preds = []
#     with torch.no_grad():
#         for image, label in tqdm(dev_loader):
#             dev_preds.append(model(image.to("cuda")))

#         dev_preds = torch.cat(dev_preds)
#         submission.append(dev_preds.cpu().numpy())
#     submission_ensembled = 0
#     for sub in submission:
#         submission_ensembled += softmax(sub, axis=1) / len(submission)
#     for columnID in range(submission_ensembled.shape[1]):
#         dev_data_cp[columnID] = submission_ensembled[:,columnID]    
#     train_data_cp.append(dev_data_cp)
# soft_labels = data[["image_id"]].merge(pd.concat(train_data_cp), how="left", on="image_id")
# soft_labels.to_csv("./tmp/soft_labels.csv", index=False)

In [21]:
soft_labels = pd.read_csv('./tmp/soft_labels.csv')
soft_labels = soft_labels.dropna()
soft_labels.reset_index(inplace=True, drop=True)

In [22]:
# soft_labels['predic_label']= soft_labels.apply(lambda x:np.argmax(x.values[3:8]),axis=1)
# soft_labels['max_prob']= soft_labels.apply(lambda x : max(x.values[3:8]),axis=1)

# soft_labels['drop'] = (soft_labels['label'] != soft_labels['predic_label']) &  (soft_labels['max_prob'] > 0.95)
# soft_labels['drop'].sum()

# clean_soft =soft_labels[~soft_labels['drop']][['image_id','label','fold','0','1','2','3','4']].reset_index(drop=True)

In [23]:
class CrossEntropyLossOneHot(nn.Module):
    def __init__(self):
        super(CrossEntropyLossOneHot, self).__init__()
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, preds, labels):
        return torch.mean(torch.sum(-labels * self.log_softmax(preds), -1))

In [24]:
import apex

In [60]:
def train_one_epoch_soft(model, optim, train_loader, loss_fn, epoch):
    model = model.train();
    t = time.time()
    running_loss = None
    preds_all = []
    targets_all = []
    loss_sum = 0
    sample_num = 0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), position=0, leave=True)
    for step, (x, y_true) in pbar:
        x = x.to(device).float()
        y_true = y_true.to(device)
        y_pred = model(x)
        l = loss_fn(y_pred, y_true.float())
        optim.zero_grad()
        if CFG['fp16']:
            with apex.amp.scale_loss(l, optim) as scaled_loss:
                scaled_loss.backward()
        else:
            l.backward()
        optim.step()
        preds_all += [torch.argmax(y_pred, 1).detach().cpu().numpy()]
        targets_all += [torch.argmax(y_true, 1).detach().cpu().numpy()]
        if running_loss is None:
            running_loss = l.item()
        else:
            running_loss = running_loss * .99 + l.item() * .01

        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
            description = f'tain epoch {epoch} loss: {running_loss:.4f}'
            pbar.set_description(description)
    
    if scheduler is not None:
        scheduler.step()
    preds_all = np.concatenate(preds_all)
    targets_all = np.concatenate(targets_all)
    print("Target acc = ", (preds_all==targets_all).mean())
    with open(LOG_FILE, 'a+') as logger:
        logger.write(f"Epoch # {epoch}, train acc = {(preds_all==targets_all).mean()}, ")

In [61]:
class Cassava_Soft_Dataset(Dataset):
    def __init__(self, df, mode = 'train'):
        super().__init__()
        self.df = df
        self.mode = mode
        
    def __len__(self, ):
        return len(self.df)
    def to_one_hot(self, le_label, num_classes = 5):
        oho_label = np.zeros(num_classes)
        oho_label[int(le_label)] = 1
        return oho_label
    
    def __getitem__(self, idx):
        row = self.df[self.df.index == idx]
        image_name = row.image_id.values[0]
        img = get_img('/home/data/Cassava/train_images/'+ image_name)
        if self.mode == 'train':
            label = self.to_one_hot(row.label.values[0])
            soft_labels = row.values[0][3:]
            label = (label*0.7).astype(float) + (soft_labels * 0.3).astype(float)
            img = TRAIN_AUGS(image=img)['image']
            #img = np.moveaxis(img, 2, 0)
            return img, label
        elif self.mode == 'val':
            label = self.to_one_hot(row.label.values[0])
            soft_labels = row.values[0][3:]
            label = (label*0.7).astype(float) + (soft_labels * 0.3).astype(float)
            img = VAL_AUGS(image=img)['image']
            return img, label
        elif self.mode == 'val_test':
            label = row.label.values[0]
            img = VAL_AUGS(image=img)['image']
            return img, label
        elif self.mode == 'test':
            img = TEST_AUGS(image=img)['image']
            return img
        else:
            print("Unknown mod type")

In [62]:
# train_data = clean_soft[(clean_soft.fold != test_fold)].reset_index(drop=True)
train_data = soft_labels[(soft_labels.fold != test_fold)].reset_index(drop=True)

val_data = data[data.fold == test_fold].reset_index(drop=True)
train_ds = Cassava_Soft_Dataset(train_data, mode = 'train')
valid_ds = Cassava_Soft_Dataset(val_data, mode = 'val_test')
    
train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=8)
    
val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=8,
        shuffle=False,
        pin_memory=False)

    
model = CassvaImgClassifier('tf_efficientnet_b4_ns', data.label.nunique(), pretrained=True).to(device)
    
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CFG['epochs'], T_mult=1, eta_min=1e-6, last_epoch=-1)
    
#loss_tr = nn.CrossEntropyLoss().to(device)
#loss_fn = nn.CrossEntropyLoss().to(device)
loss_tr = loss_fn = CrossEntropyLossOneHot().to(device)
loss_val = nn.CrossEntropyLoss().to(device)

if CFG['fp16']:
    model, optimizer = apex.amp.initialize(
        model,
        optimizer,
        opt_level='O1')

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [63]:
for epoch in range(CFG['epochs']):
    print(f"epoch number {epoch}, lr = {optimizer.param_groups[0]['lr']}")
    train_one_epoch_soft(model, optimizer, train_loader, loss_tr, epoch)
    val_acc, val_loss = valid_one_epoch(model, optimizer, val_loader, loss_val, epoch)

    #torch.save(model.state_dict(),'./output/{}_soft_dev_fold_{}_test_fold_{}_epoch_{}_val_loss_{:.4f}_val_acc_{:.4f}'.format(CFG['model_arch'], dev_fold, test_fold, epoch, val_acc, val_loss))

#del model, optimizer, train_loader, val_loader, scaler, scheduler
torch.cuda.empty_cache()

epoch number 0, lr = 0.0001


/cm/shared/apps/jupyterhub/0.8.1/lib/python3.6/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':



Target acc =  0.8148039960273412


/cm/shared/apps/jupyterhub/0.8.1/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



validation multi-class accuracy = 0.8654
epoch number 1, lr = 9.757729755661011e-05


...


validation multi-class accuracy = 0.8829
epoch number 8, lr = 1.0453658778440109e-05



Target acc =  0.9112578138692528



validation multi-class accuracy = 0.8860
epoch number 9, lr = 3.4227024433899e-06



Target acc =  0.9140620435824035



validation multi-class accuracy = 0.8839
